In [14]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

In [15]:
from tqdm import tqdm
from pathlib import Path
from src.utils.config_loader import load_config
from src.utils.seed import seed_everything

base_dir = Path(os.getcwd()).parent

config = load_config(base_dir / 'secrets.yaml')

seed_everything(42)

In [16]:
from src.data.preprocessing import create_df

val_df = create_df(base_dir / 'data/my_data/regplans-dev.conllu')

In [17]:
from langchain_openai import AzureChatOpenAI
from langchain_core.messages import (SystemMessage, HumanMessage)

os.environ['OPENAI_API_VERSION'] = config['OPENAI_API_VERSION']
os.environ['AZURE_OPENAI_ENDPOINT'] = config['OPENAI_API_BASE']
os.environ['AZURE_OPENAI_API_KEY'] = config['OPENAI_API_KEY']

llm = AzureChatOpenAI(
    deployment_name='o1-mini',
    #temperature=0.0
)

In [18]:
import json

def format_examples(example_subset): 
    # Formats the examples into a string for later prompt
    formatted = []
    for i, ex in enumerate(example_subset):
        entity_lines = "\n".join([f"{e['word']} {e['label']}" for e in ex["entities"]])
        formatted.append(f"Eksempel {i+1}:\nTekst: \"{ex['sentence']}\"\nEntiteter:\n{entity_lines}\n##\n")
    
    return "\n".join(formatted)

with open(base_dir / 'llm_stuff/prompts/examples.json', 'r') as f:
    example_bank = json.load(f)

ids = [19, 16, 3]

examples = [next(ex for ex in example_bank if ex["id"] == id) for id in ids]

formatted_examples = format_examples(examples)

print(formatted_examples)

Eksempel 1:
Tekst: "Parkeringsplassar ( SPP ) Grøntstruktur , jf . PBL § 12-5 , 2 . ledd nr . 3 - Turveg ( GT )"
Entiteter:
SPP B-FELT
GT B-FELT
##

Eksempel 2:
Tekst: "Før det vert gjeve mellombels bruksløyve / ferdigattest for ny bueining innanfor felt BKS1 og BFS14 og 15"
Entiteter:
BKS1 B-FELT
BFS14 B-FELT
og I-FELT
15 I-FELT
##

Eksempel 3:
Tekst: "Bebyggelsestype Innenfor BKS1-BKS6 og BFS2 skal det oppføres flermannsboliger , kjedeboliger og / eller rekkehus ."
Entiteter:
BKS1-BKS6 B-FELT
BFS2 B-FELT
##



In [19]:
from src.utils.label_mapping_regplans import label_to_id
from collections import defaultdict

all_pred_ids = []
all_true_ids = []
all_results = []

val_df = val_df.iloc[:int(len(val_df) * 0.5)]

for idx, row in tqdm(val_df.iterrows(), total=len(val_df)):
    sentence = row['full_text']
    tokens = row['words']
    true_labels = row['labels']

    prompt = f"""
    Du er en ekspert på Named Entity Recognition (NER). Din oppgave er å identifisere entiteter som representerer feltnavn i tekstutdrag fra reguleringsplaner.
    
    De eneste gyldige etikettene er B-FELT (begynnelsen på et feltnavn) og I-FELT (fortsettelsen av det samme feltnavnet).

    {formatted_examples}

    Formuler svaret over flere linjer, en token per linje, men kun tokens som inngår i et feltnavn. Hver linje skal inneholde tokenet etterfulgt av tilhørende etikett, atskilt med ett mellomrom.
                 
    Tekst: '{sentence}'

    Entiteter:
    """
    
    msg = [HumanMessage(content=prompt)]

    try:
        response = llm.invoke(msg)
        entities = defaultdict(list)

        for line in response.content.splitlines():
            parts = line.strip().split()
            if len(parts) == 2:
                word, label = parts[0], parts[1]
                entities[word].append(label)

        pred_labels = []
        word_counts = defaultdict(int)

        for token in tokens:
            if token in entities and word_counts[token] < len(entities[token]):
                pred_labels.append(entities[token][word_counts[token]])
                word_counts[token] += 1
            else:
                pred_labels.append("O")

        pred_ids = [label_to_id.get(label, label_to_id.get("O", -1)) for label in pred_labels]
        true_ids = [label_to_id[label] for label in true_labels]

        all_pred_ids.extend(pred_ids)
        all_true_ids.extend(true_ids)

        all_results.append({
            'sentence': sentence,
            'tokens': tokens,
            'true_labels': true_labels,
            'predicted_labels': pred_labels,
            'generated_text': response.content
        })

    except Exception as e:
        print(f"Skipping row {idx} due to error: {e}")
        continue


  0%|          | 0/176 [00:00<?, ?it/s]

100%|██████████| 176/176 [23:28<00:00,  8.00s/it]


In [20]:
from llm_stuff.evaluation import evaluate 

metrics = evaluate(all_true_ids, all_pred_ids)

print("Evaluation Metrics on Dev Set:")
print(metrics)

final_output = {
    'prompt': str(msg),
    'evaluation_metrics': metrics,
    'results': all_results
}

with open(base_dir / f"llm_stuff/results/o1-mini_FEWSHOT_3_Norwegian.json", 'w', encoding='utf-8') as f:
    json.dump(final_output, f, indent=4, ensure_ascii=False)

Evaluation Metrics on Dev Set:
{'precision': 0.8739615678787231, 'recall': 0.5719550848007202, 'f1': 0.6202458143234253, 'span_acc': 0.48427674174308777, 'classification_report': {'B-FELT': {'precision': 0.8333333333333334, 'recall': 0.5974842767295597, 'f1-score': 0.6959706959706959, 'support': 159.0}, 'I-FELT': {'precision': 0.875, 'recall': 0.11965811965811966, 'f1-score': 0.21052631578947367, 'support': 117.0}, 'O': {'precision': 0.9135514018691588, 'recall': 0.9987228607918263, 'f1-score': 0.9542403904820013, 'support': 1566.0}, 'accuracy': 0.9082519001085776, 'macro avg': {'precision': 0.8739615784008308, 'recall': 0.5719550857265019, 'f1-score': 0.6202458007473903, 'support': 1842.0}, 'weighted avg': {'precision': 0.9041783362253544, 'recall': 0.9082519001085776, 'f1-score': 0.8847075847456695, 'support': 1842.0}}}
